In [1]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacitydemoml2
Azure region: eastus2
Subscription id: 9d855d0b-cba0-4253-aa5d-91f6e6ebc1d5
Resource group: udacitycvdemo


# SK-Learn

## Setup Cluster for sklearn training

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "ml-project-nd00333"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-09-28T09:52:25.057000+00:00', 'errors': None, 'creationTime': '2024-09-28T04:00:44.715513+00:00', 'modifiedTime': '2024-09-28T04:00:54.729448+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'Standard_D2_v2'}


## Setup Hyper parameter tuning for sklearn

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--max_iter': choice(50, 100, 200),
        '--C': uniform(0.01, 10)
    }
)

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='.', 
                      script='train.py', 
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(hyperdrive_config)

In [5]:
run.wait_for_completion(show_output=True)

RunId: HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66
Web View: https://ml.azure.com/runs/HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66?wsid=/subscriptions/9d855d0b-cba0-4253-aa5d-91f6e6ebc1d5/resourcegroups/udacitycvdemo/workspaces/udacitydemoml2&tid=c50d08fb-64e9-430f-b8fd-e1eae4d0b61c

Streaming azureml-logs/hyperdrive.txt

[2024-09-28T10:09:43.407323][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-09-28T10:09:43.8522516Z][SCHEDULER][INFO]Scheduling job, id='HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66_0' 
[2024-09-28T10:09:43.9898554Z][SCHEDULER][INFO]Scheduling job, id='HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66_1' 
[2024-09-28T10:09:44.1774160Z][SCHEDULER][INFO]Scheduling job, id='HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66_3' 
[2024-09-28T10:09:44.1024656Z][SCHEDULER][INFO]Scheduling job, id='HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66_2' 
[2024-09-28T10:09:44.137179][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024-09-2

{'runId': 'HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66',
 'target': 'ml-project-nd00333',
 'status': 'Completed',
 'startTimeUtc': '2024-09-28T10:09:42.545647Z',
 'endTimeUtc': '2024-09-28T10:28:22.422717Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'a6bba97a-cbef-4182-9244-81ea5f04e5e7',
  'user_agent': 'python/3.9.19 (Linux-5.15.0-1064-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66_10',
  'score': '0.9088012139605464',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ec98bade-a6c5-4a06-a4f1-31b5aaac2e66_10'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'tel

In [6]:
best_run = run.get_best_run_by_primary_metric()
hpo_metrics = best_run.get_metrics()
print(hpo_metrics)

{'Regularization Strength:': 6.092621988412589, 'Max iterations:': 100, 'Accuracy': 0.9088012139605463}


In [7]:
# Get your best run and save the model from that run.
best_model_name = [x for x in best_run.get_file_names() if '.joblib' in x][0]
best_run.download_file(name=best_model_name, output_file_path='./training')


# AutoML

## Extract Data

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
csv_pth = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=csv_pth)

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

train_df = pd.concat([x_train,y_train], axis=1)

train_df.to_csv('train.csv', index=False)

# set datastore
datastore = ws.get_default_datastore()

# register dataset from pandas dataframe
dataset = Dataset.Tabular.register_pandas_dataframe(
    dataframe=train_df, 
    target=(datastore, 'train.csv'),  
    name='bankmarketing_train_automl' 
)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
Validating arguments.
Arguments validated.
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data

## Setup and run AutoML

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target)

In [11]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on ml-project-nd00333 with default configuration
Running on remote compute: ml-project-nd00333


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_71520427-47ea-41b8-b176-7733096b7c9b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

## Save model and Metrics for AutoML

In [13]:
# Retrieve and save your best automl model.
import joblib

### YOUR CODE HERE ###
best, model = automl_run.get_output()
joblib.dump(model, './training/bestAutoMl.joblib')

['./training/bestAutoMl.joblib']

In [15]:
# show metrics
metrics = best.get_metrics()
print(metrics['accuracy'])

0.9183232169954477


In [28]:
for x in metrics.keys():
    print(f"{x}: {metrics[x]:.3}")

precision_score_macro: 0.799
accuracy: 0.918
recall_score_micro: 0.918
precision_score_micro: 0.918
precision_score_weighted: 0.915
average_precision_score_weighted: 0.956
balanced_accuracy: 0.776
log_loss: 0.303
weighted_accuracy: 0.954
recall_score_macro: 0.776
average_precision_score_macro: 0.827
f1_score_macro: 0.786
AUC_macro: 0.948
recall_score_weighted: 0.918
matthews_correlation: 0.574
f1_score_micro: 0.918
AUC_micro: 0.981
AUC_weighted: 0.948
norm_macro_recall: 0.551
average_precision_score_micro: 0.982
f1_score_weighted: 0.917
confusion_matrix: aml
accuracy_table: aml
